<a href="https://colab.research.google.com/github/cauekssouza/Mem-ria/blob/main/Memoria_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install memory_profiler

In [ ]:
import numpy as np
import time
import memory_profiler
import matplotlib.pyplot as plt
import sys
import gc
from decimal import Decimal, getcontext

# Configura a precisão para 100 casas decimais
getcontext().prec = 102

def medir_memoria_tempo(func, *args):
    gc.collect()  # Força a coleta de lixo antes da medição
    memoria_antes = memory_profiler.memory_usage()[0]
    tempo_inicio = time.perf_counter_ns()
    resultado = func(*args)
    tempo_fim = time.perf_counter_ns()
    memoria_depois = memory_profiler.memory_usage()[0]
    memoria_usada = memoria_depois - memoria_antes
    return resultado, (tempo_fim - tempo_inicio), memoria_usada

def criar_array_aleatorio():
    return np.random.choice(list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"), size=10000000)

def criar_lista_aleatoria():
    return list(np.random.choice(list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"), size=10000000))

def ordenar_array(arr):
    return np.sort(arr)

def ordenar_lista(lst):
    return sorted(lst)

def imprimir_array(arr):
    sys.stdout = open('/dev/null', 'w')  # Suprime a saída para medir apenas tempo/memória
    print(arr)
    sys.stdout.close()
    sys.stdout = sys.__stdout__  # Restaura a saída padrão

def imprimir_lista(lst):
    sys.stdout = open('/dev/null', 'w')  # Suprime a saída para medir apenas tempo/memória
    print(lst)
    sys.stdout.close()
    sys.stdout = sys.__stdout__  # Restaura a saída padrão

def calcular_pi(n_termos=1000000):
    pi = Decimal(0)
    for k in range(n_termos):
        pi += Decimal((-1)**k) / Decimal(2*k + 1)
    pi *= 4
    return pi

dados = {}

# Medições para arrays NumPy
dados['criar_array'] = medir_memoria_tempo(criar_array_aleatorio)
dados['ordenar_array'] = medir_memoria_tempo(ordenar_array, dados['criar_array'][0])
dados['imprimir_array'] = medir_memoria_tempo(imprimir_array, dados['criar_array'][0])

# Medições para listas Python
dados['criar_lista'] = medir_memoria_tempo(criar_lista_aleatoria)
dados['ordenar_lista'] = medir_memoria_tempo(ordenar_lista, dados['criar_lista'][0])
dados['imprimir_lista'] = medir_memoria_tempo(imprimir_lista, dados['criar_lista'][0])

# Medição para cálculo de π
dados['calcular_pi'] = medir_memoria_tempo(calcular_pi)

# Exibição dos resultados
for chave, (_, tempo_ns, memoria_dif) in dados.items():
    print(f"{chave}: Tempo = {tempo_ns / 1e6:.3f} ms, Memória = {memoria_dif:.6f} MiB")

# Gráficos comparativos
tarefas = ["Criar", "Ordenar", "Imprimir"]
valores_tempo = [dados[f'criar_array'][1], dados[f'ordenar_array'][1], dados[f'imprimir_array'][1],
                 dados[f'criar_lista'][1], dados[f'ordenar_lista'][1], dados[f'imprimir_lista'][1]]
valores_tempo = [t / 1e6 for t in valores_tempo]  # Convertendo para milissegundos

valores_memoria = [dados[f'criar_array'][2], dados[f'ordenar_array'][2], dados[f'imprimir_array'][2],
                   dados[f'criar_lista'][2], dados[f'ordenar_lista'][2], dados[f'imprimir_lista'][2]]

rotulos = ["Array (Criar)", "Array (Ordenar)", "Array (Imprimir)",
           "Lista (Criar)", "Lista (Ordenar)", "Lista (Imprimir)"]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.bar(rotulos, valores_tempo, color=['blue', 'blue', 'blue', 'orange', 'orange', 'orange'])
plt.ylabel("Tempo (ms)")
plt.title("Tempo de execução")
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar(rotulos, valores_memoria, color=['blue', 'blue', 'blue', 'orange', 'orange', 'orange'])
plt.ylabel("Memória (MiB)")
plt.title("Uso de Memória")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()
